In [ ]:
from auth import app_id, user_login, user_password
import vk

class ApiConnection:
    
    def __init__(self, _app_id=app_id, _user_login=user_login, _user_password=user_password):
        self._app_id = _app_id
        self._user_login = _user_login
        self._user_password = _user_password
        
    def get_api(self):       
        session = vk.AuthSession(self._app_id, 
                                 self._user_login, 
                                 self._user_password, 
                                 scope='messages')
        return vk.API(session)
    
vk_api = ApiConnection().get_api()

In [ ]:
def add_message_to_db(message):
    print (message)
    print ("++++++++++")
    
def messages_generator(dialog_id, step=100):
    offset = 0
    hist = vk_api.messages.getHistory(peer_id=str(dialog_id), user_id=my_id, count=step, offset=offset)
    while len(hist['items']) > 0:
        for item in hist['items']:  
            yield item  
        offset += step

        hist = vk_api.messages.getHistory(peer_id=str(dialog_id), user_id=my_id, count=step, offset=offset)    

In [ ]:
def dialogs_generator(step=100):
    offset = 0
    diags = vk_api.messages.getDialogs(offset=offset, count=step)
    while len(diags['items']) > 0:
        for item in diags['items']:  
            try:
                yield 2000000000 + item['message']['chat_id']
            except KeyError as e:
                if 'chat_id' in e.args:
                    yield item['message']['user_id']
                else:
                    raise
            except:
                raise
        offset += step
        diags = vk_api.messages.getDialogs(offset=offset, count=step)

In [ ]:
import time
d_counter = 1

for dialog_id in dialogs_generator(1):
    time.sleep(1)
    if d_counter > 3:
        break
    d_counter += 1
    m_counter = 1
    for message in messages_generator(dialog_id=dialog_id, step=1):
        if m_counter > 3:
            break
        m_counter += 1
        add_message_to_db(message)